In [1]:
import pandas as pd
from datetime import datetime

In [2]:
data = pd.read_json('sherdog/data/cumulative.json')

In [31]:
def parse_date(timeinmilis):
    try:
        timestamp = timeinmilis / 1000
        return datetime.utcfromtimestamp(timestamp)
    except:
        return None

    
def flatten(data):
    X = pd.DataFrame()
    X['date'] = data['date']
    X['target'] = data['result'].map({'win': 1, 'loss': 0})
    for fighter in ['fighter', 'opponent']:
        for key in ['birth', 'started']:
            curr = '{}_{}'.format(fighter, key)
            X[curr] = data[fighter].apply(lambda x: parse_date(x[key]))

        # Cumulative stats
        for key in ['loss', 'win']:
            stats = 'cumulative'
            for result in ['loss', 'win']:
                for method in ['decision', 'knockout', 'submission', 'total']:
                    curr = '{}_{}_{}_{}_{}'.format(fighter, stats, key, result, method)
                    X[curr] = data[fighter].apply(lambda x: x[stats][key][result][method])

        # Historical stats
        stats = 'history'
        for result in ['loss', 'win']:
            for method in ['decision', 'knockout', 'submission', 'total']:
                curr = '{}_{}_{}_{}'.format(fighter, stats, result, method)
                X[curr] = data[fighter].apply(lambda x: x[stats][result][method])

        for key in ['fights', 'positions', 'time']:
            curr = '{}_{}_{}'.format(fighter, stats, key)
            X[curr] = data[fighter].apply(lambda x: x[stats][key])
    
    return X


In [32]:
x = flatten(data)

In [33]:
x.head()

,date,target,fighter_birth,fighter_started,fighter_cumulative_loss_loss_decision,fighter_cumulative_loss_loss_knockout,fighter_cumulative_loss_loss_submission,fighter_cumulative_loss_loss_total,fighter_cumulative_loss_win_decision,fighter_cumulative_loss_win_knockout,...,opponent_history_loss_knockout,opponent_history_loss_submission,opponent_history_loss_total,opponent_history_win_decision,opponent_history_win_knockout,opponent_history_win_submission,opponent_history_win_total,opponent_history_fights,opponent_history_positions,opponent_history_time
0,2017-04-22,1,1990-09-02,2013-06-01,1.0,0.0,2.0,3.0,2.0,9.0,...,5.0,0.0,7.0,1.0,3.0,4.0,8.0,15.0,60.0,26.633333
1,2017-04-22,0,1985-08-09,2010-05-01,13.0,33.0,3.0,49.0,11.0,16.0,...,0.0,1.0,2.0,2.0,7.0,1.0,10.0,12.0,51.0,33.883333
10,2010-10-22,1,1978-07-21,2012-05-05,1.0,0.0,1.0,2.0,1.0,2.0,...,1.0,2.0,3.0,0.0,3.0,0.0,3.0,6.0,23.0,8.683333
100,2011-09-30,0,1993-03-10,2011-04-02,2.0,1.0,1.0,4.0,2.0,11.0,...,1.0,1.0,4.0,1.0,6.0,8.0,15.0,19.0,61.0,59.933333
1000,2016-05-22,0,1988-02-18,2007-04-01,11.0,0.0,0.0,11.0,11.0,30.0,...,0.0,1.0,1.0,3.0,2.0,3.0,8.0,9.0,59.0,31.183333
